In [1]:
import pandas as pd
import seaborn as sns
import re
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, roc_curve, auc
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
print(transformers.__version__)

W0901 17:18:23.963000 95112 site-packages/torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


4.53.3


# (Thai) OCR Test through tesserocr 

In [2]:
import os
from pathlib import Path
from pdf2image import convert_from_path
from PIL import Image, ImageFilter, ImageOps
import tesserocr
import PyPDF2
import Levenshtein
import re
import cv2
import numpy as np
import easyocr
import tesserocr

In [3]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [4]:
# === Config ===
pdf_path = "ExampleOCR/ocr_example_1.pdf"
poppler_path = "/opt/homebrew/bin"
tessdata_dir = "/opt/homebrew/share/tessdata"
pic_dpi = 300
max_pages = 3
use_erosion = False

In [5]:
output_dir = Path(pdf_path).parent
pdf_stem = Path(pdf_path).stem

In [6]:
# === Extract Ground Truth ===
ground_truth_text = []
with open(pdf_path, "rb") as f:
    reader = PyPDF2.PdfReader(f)
    for i, page in enumerate(reader.pages):
        if max_pages and i >= max_pages:
            break
        text = page.extract_text() or ""
        ground_truth_text.append(text.strip())

groundtruth_output_path = output_dir / f"{pdf_stem}_groundtruth.txt"
with open(groundtruth_output_path, "w", encoding="utf-8") as gt_file:
    for i, gt in enumerate(ground_truth_text):
        gt_file.write(f"\n--- Page {i + 1} ---\n{gt}\n")
print(f"Ground truth saved to: {groundtruth_output_path.name}")

Ground truth saved to: ocr_example_1_groundtruth.txt


In [7]:
# === Convert PDF to images ===
pages = convert_from_path(pdf_path, dpi=pic_dpi, poppler_path=poppler_path)
if max_pages:
    pages = pages[:max_pages]

In [8]:
# === Image Preprocessing Function ===
def preprocess_image(image, use_erosion=False):
    """
    Preprocess a PIL image for OCR by enhancing contrast, sharpening,
    binarizing, and optionally applying erosion.

    Args:
        image (PIL.Image.Image): Input image.
        use_erosion (bool): Whether to apply erosion filter (3x3 min pooling).

    Returns:
        PIL.Image.Image: Processed binary image.
    """
    # Convert to grayscale
    gray = image.convert('L')

    # Auto-contrast enhancement
    contrast = ImageOps.autocontrast(gray)

    # Apply unsharp masking
    sharpened = contrast.filter(ImageFilter.UnsharpMask(radius=2, percent=150, threshold=3))

    # Convert to NumPy array for binarization
    img_np = np.array(sharpened).astype(np.uint8)
    binary = (img_np > 160).astype(np.uint8) * 255  # Manual thresholding

    # Optional erosion (3x3 minimum filter)
    if use_erosion:
        padded = np.pad(binary, pad_width=1, mode='edge')
        eroded = np.zeros_like(binary)
        for i in range(eroded.shape[0]):
            for j in range(eroded.shape[1]):
                eroded[i, j] = np.min(padded[i:i+3, j:j+3])
        binary = eroded

    # Convert back to PIL image
    return Image.fromarray(binary.astype(np.uint8), mode='L')


# === Text Cleaning ===
def remove_garbage_lines(text):
    lines = text.splitlines()
    return "\n".join([line for line in lines if len(re.sub(r"[^\u0E00-\u0E7F]", "", line)) > 5])

def clean_ocr_text(text):
    text = re.sub(r"[^\u0E00-\u0E7F\s0-9a-zA-Z๐-๙.,“”()\"'ฯ\-/:]", "", text)
    text = re.sub(r"\s{2,}", " ", text)
    return text.strip()

In [9]:
def run_ocr_pipeline(
    pages, output_dir, pdf_stem, groundtruth_output_path,
    ground_truth_text, ocr_engine="tesserocr", tessdata_dir="", use_erosion=False
):
    """
    Perform OCR on a list of image pages using either Tesserocr or EasyOCR,
    save the processed images and extracted text, and compute accuracy if ground truth is available.

    Args:
        pages (List[PIL.Image.Image]): List of PIL Image pages to process.
        output_dir (Path): Directory where processed images will be saved.
        pdf_stem (str): Base name of the PDF for output file naming.
        groundtruth_output_path (str): Path to save the ground truth text (not used in writing, just for info).
        ground_truth_text (List[str]): List of ground truth strings per page.
        ocr_engine (str): OCR engine to use, either "tesserocr" or "easyocr".
        tessdata_dir (str): Directory path to Tesseract language data files (used if ocr_engine is "tesserocr").
        use_erosion (bool): Whether to apply erosion during preprocessing.

    Returns:
        None
    """
    # === OCR and Save ===
    ocr_output_path = output_dir / f"{pdf_stem}_ocr_{ocr_engine}_output.txt"
    accuracy_results = []

    accuracy_results = []
    reader = easyocr.Reader(['th'], gpu=False) if ocr_engine == "easyocr" else None

    with open(ocr_output_path, "w", encoding="utf-8") as out:
        if ocr_engine == "tesserocr":
            with tesserocr.PyTessBaseAPI(path=tessdata_dir, lang='tha', oem=tesserocr.OEM.LSTM_ONLY) as api:
                api.SetPageSegMode(tesserocr.PSM.AUTO)
                for i, page in enumerate(pages):
                    image_path = output_dir / f"{pdf_stem}_page_{i + 1}.png"
                    processed = preprocess_image(page, use_erosion=use_erosion)
                    processed.save(image_path)
                    print(f"Saved image: {image_path.name}")

                    api.SetImage(processed)
                    raw_ocr = api.GetUTF8Text()
                    ocr_text = clean_ocr_text(raw_ocr)
                    ocr_text = remove_garbage_lines(ocr_text)

                    out.write(f"\n--- Page {i + 1} ---\n{ocr_text}\n")

                    gt_text = ground_truth_text[i] if i < len(ground_truth_text) else ""
                    if gt_text:
                        similarity = Levenshtein.ratio(ocr_text, gt_text)
                        accuracy_results.append((i + 1, similarity))
                        print(f"Page {i + 1} accuracy: {similarity:.2%}")
                    else:
                        print(f"⚠️ No ground truth available for page {i + 1}")

        elif ocr_engine == "easyocr":
            for i, page in enumerate(pages):
                image_path = output_dir / f"{pdf_stem}_page_{i + 1}.png"
                processed = preprocess_image(page, use_erosion=use_erosion)
                processed.save(image_path)
                print(f"Saved image: {image_path.name}")

                img_rgb = np.array(processed.convert("RGB"), dtype=np.uint8)
                img_bgr = np.ascontiguousarray(img_rgb[:, :, ::-1], dtype=np.uint8)

                print(f"Shape: {img_bgr.shape}, Dtype: {img_bgr.dtype}, Contiguous: {img_bgr.flags['C_CONTIGUOUS']}")
                _ = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)  # Ensure valid format

                results = reader.readtext(img_bgr, detail=0, paragraph=True)
                raw_ocr = "\n".join(results)
                ocr_text = clean_ocr_text(raw_ocr)
                # ocr_text = remove_garbage_lines(ocr_text)  # Uncomment if needed

                out.write(f"\n--- Page {i + 1} ---\n{ocr_text}\n")

                gt_text = ground_truth_text[i] if i < len(ground_truth_text) else ""
                if gt_text:
                    similarity = Levenshtein.ratio(ocr_text, gt_text)
                    accuracy_results.append((i + 1, similarity))
                    print(f"Page {i + 1} accuracy: {similarity:.2%}")
                else:
                    print(f"⚠️ No ground truth available for page {i + 1}")

    print("\n==== OCR Summary: ====")
    for page_num, acc in accuracy_results:
        print(f"  - Page {page_num}: {acc:.2%} match")
    print(f"\nOCR output saved to: {ocr_output_path}")
    print(f"Ground truth saved to: {groundtruth_output_path}")

In [10]:
ocr_engine = "tesserocr" 
run_ocr_pipeline(
    pages, output_dir, pdf_stem, groundtruth_output_path,
    ground_truth_text, ocr_engine=ocr_engine, tessdata_dir=tessdata_dir, use_erosion=use_erosion
)

Saved image: ocr_example_1_page_1.png
Page 1 accuracy: 82.67%
Saved image: ocr_example_1_page_2.png
Page 2 accuracy: 93.58%
Saved image: ocr_example_1_page_3.png
Page 3 accuracy: 94.78%

==== OCR Summary: ====
  - Page 1: 82.67% match
  - Page 2: 93.58% match
  - Page 3: 94.78% match

OCR output saved to: ExampleOCR/ocr_example_1_ocr_tesserocr_output.txt
Ground truth saved to: ExampleOCR/ocr_example_1_groundtruth.txt


In [ ]:
ocr_engine = "easyocr" 
run_ocr_pipeline(
    pages, output_dir, pdf_stem, groundtruth_output_path,
    ground_truth_text, ocr_engine=ocr_engine, tessdata_dir=tessdata_dir, use_erosion=use_erosion
)

Saved image: ocr_example_1_page_1.png
Shape: (3509, 2481, 3), Dtype: uint8, Contiguous: True


# Both OCR result combined through Gemini Flash 2.5
หน้า ๑๑
เล่ม ๑๓๔ ตอนที่ ๒๔ ก.
ราชกิจจานุเบกษา
๒๔ กุมภาพันธ์ ๒๕๖๐

พระราชบัญญัติ
การจัดซื้อจัดจ้างและการบริหารพัสดุภาครัฐ พ.ศ. ๒๕๖๐

สมเด็จพระเจ้าอยู่หัวมหาวชิราลงกรณ บดินทรเทพยวรางกูร ให้ไว้ ณ วันที่ ๒๔ กุมภาพันธ์ พ.ศ. ๒๕๖๐ เป็นปีที่ ๒ ในรัชกาลปัจจุบัน

สมเด็จพระเจ้าอยู่หัวมหาวชิราลงกรณ บดินทรเทพยวรางกูร มีพระราชโองการโปรดเกล้าฯ ให้ประกาศว่า โดยที่เป็นการสมควรมีกฎหมายว่าด้วยการจัดซื้อจัดจ้างและการบริหารพัสดุภาครัฐ จึงทรงพระกรุณาโปรดเกล้าฯ ให้ตราพระราชบัญญัติขึ้นไว้โดยคำแนะนำและยินยอมของ สภานิติบัญญัติแห่งชาติ ดังต่อไปนี้

มาตรา ๑ พระราชบัญญัตินี้เรียกว่า “พระราชบัญญัติการจัดซื้อจัดจ้างและการบริหารพัสดุภาครัฐ พ.ศ. ๒๕๖๐”

มาตรา ๒ พระราชบัญญัตินี้ให้ใช้บังคับเมื่อพ้นกำหนดหนึ่งร้อยแปดสิบวันนับแต่วันประกาศในราชกิจจานุเบกษาเป็นต้นไป

มาตรา ๓ ให้ยกเลิกบทบัญญัติเกี่ยวกับพัสดุ การจัดซื้อจัดจ้าง หรือการบริหารพัสดุ ในกฎหมาย ระเบียบ ข้อบังคับ ประกาศ ข้อบัญญัติ และข้อกำหนดใด ๆ ของหน่วยงานของรัฐที่อยู่ภายใต้บังคับแห่งพระราชบัญญัตินี้

มาตรา ๔ ในพระราชบัญญัตินี้ “การจัดซื้อจัดจ้าง” หมายความว่า การดำเนินการเพื่อให้ได้มาซึ่งพัสดุโดยการซื้อ จ้าง เช่า แลกเปลี่ยน หรือโดยนิติกรรมอื่นตามที่กำหนดในกฎกระทรวง

“พัสดุ” หมายความว่า สินค้า งานบริการ งานก่อสร้าง งานจ้างที่ปรึกษาและงานจ้างออกแบบ หรือควบคุมงานก่อสร้าง รวมทั้งการดำเนินการอื่นตามที่กำหนดในกฎกระทรวง